In [1]:
from keras.models import load_model, Model
from keras import backend as K
import pandas as pd
import numpy as np
from keras.layers import Input

Using TensorFlow backend.


In [8]:
test_df = pd.read_csv('../dataset/mnist-in-csv/mnist_test.csv')
m_test = len(test_df)
X_test = np.array(test_df.loc[:, test_df.columns != 'label'].values.reshape(m_test, 28, 28, 1), dtype=np.float64)
y_test = test_df['label'].values
X_test /= 255

In [3]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [3]:
model = load_model('MNIST-triplet-loss-network.h5', compile=False)

In [5]:
input_layer = Input((28,28,1))
extracted_network = model.get_layer(index=3)
extracted_network = extracted_network(input_layer)

In [6]:
model = Model(input_layer, extracted_network)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 4)                 2508      
Total params: 2,508
Trainable params: 2,508
Non-trainable params: 0
_________________________________________________________________


In [9]:
y_test[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4])

In [29]:
image_a = X_test[2]
image_a = np.expand_dims(image_a, axis=0)
image_a_encoding = model.predict(image_a)
print(image_a_encoding.shape)

image_b = X_test[5]
image_b = np.expand_dims(image_b, axis=0)
image_b_encoding = model.predict(image_b)
print(image_b_encoding.shape)

print(image_a_encoding)
print(image_b_encoding)

dist = np.linalg.norm(image_a_encoding - image_b_encoding)
print(dist)

(1, 4)
(1, 4)
[[-0.04856872  0.5303569   0.23984893  0.45753506]]
[[-0.10379883  0.5909315   0.10714564  0.47095898]]
0.15655674


In [26]:
model.save_weights('trained-weights-model/triplet_loss_encoding_network_weights.h5')
model.save('trained-weights-model/triplet_loss_encoding_network_model.h5')